### Özellik Mühendisliği & Veri Ön İşleme(Feature Engineering & Data Preprocessing)

- Eğer veriniz kötüyse makine öğrenim araçlarınız kullanışlı olmayacaktır.
- Dünyanın en önemli kaynağı petrol değil veridir.
Özellik mühendisliği: Özellik üzerinde gerçekleştirilen çalışmalar ve ham veriden değişken üretmektir.
Veri Ön İşleme: Çalışmalar öncesi verinin uygun hale getirilmesi sürecidir.

### OUTLIERS(Aykırı Değerler)
- Verideki genel eğilimin oldukça dışına çıkan değerlere **aykırı değer** denir.
- Özellikle doğrusal problemlerde aykırı değerlerin şiddeti oldukça güçlüdür.
- Ağaç yöntemlerinde bu etkiler daha düşüktür.